# 0) Setup

In [5]:
# ---- Setup for ALL Notebooks ---
from proj.setup import setup_notebook
setup_notebook() # sets up paths, autoreload, etc.

# --- Setup for SQL Notebooks ---
from proj.db import connect_to_db, close_db

# --- Connect to SQLite database ---
conn, cursor = connect_to_db()

# test query
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", cursor.fetchall())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Notebook setup complete.
Project root: /Users/b-rad.j.neiman/CODE/diabetes-capstone
✅ Connected to database: /Users/b-rad.j.neiman/CODE/diabetes-capstone/data/diabetes.db
Tables: [('diabetes_patients',)]


# 1) Utility: run a query and print results (no pandas)

In [6]:
from textwrap import shorten

def run_sql(query, params=None, limit_preview=10):
    params = params or ()
    cursor.execute(query, params)
    rows = cursor.fetchall()
    cols = [d[0] for d in cursor.description] if cursor.description else []
    
    # pretty print
    print("SQL:", query.strip(), "\n")
    if not cols:
        print(f"Done. {cursor.rowcount if cursor.rowcount != -1 else 'OK'}")
        return rows
        
    # header
    print(" | ".join(cols))
    print("-" * min(120, 4*len(" | ".join(cols))))
    
    # rows (preview)
    for r in rows[:limit_preview]:
        print(" | ".join(str(x) for x in r))
    if len(rows) > limit_preview:
        print(f"... ({len(rows)-limit_preview} more rows)")
    return rows

# 2) Examples

## a) Count

In [7]:
rows = run_sql("SELECT COUNT(*) AS n_rows FROM diabetes_patients;")
n_rows = rows[0][0]
print("\nCount of patients:", n_rows)

SQL: SELECT COUNT(*) AS n_rows FROM diabetes_patients; 

n_rows
------------------------
768

Count of patients: 768


## b) Parameterized example

In [8]:
run_sql("""
SELECT outcome, COUNT(*) AS n
FROM diabetes_patients
GROUP BY outcome
ORDER BY n DESC;
""")

SQL: SELECT outcome, COUNT(*) AS n
FROM diabetes_patients
GROUP BY outcome
ORDER BY n DESC; 

Outcome | n
--------------------------------------------
0 | 500
1 | 268


[(0, 500), (1, 268)]

## c) Filter + Aggregate by range

In [9]:
run_sql("""
SELECT 
    outcome, 
    COUNT(*) AS N,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM diabetes_patients), 2) AS pct_of_population,
    ROUND(AVG(bmi),2) AS avg_bmi, 
    ROUND(AVG(age), 2) AS avg_age
FROM diabetes_patients
WHERE bmi >= 30
GROUP BY outcome;
""")

SQL: SELECT 
    outcome, 
    COUNT(*) AS N,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM diabetes_patients), 2) AS pct_of_population,
    ROUND(AVG(bmi),2) AS avg_bmi, 
    ROUND(AVG(age), 2) AS avg_age
FROM diabetes_patients
WHERE bmi >= 30
GROUP BY outcome; 

Outcome | N | pct_of_population | avg_bmi | avg_age
------------------------------------------------------------------------------------------------------------------------
0 | 253 | 32.94 | 36.04 | 31.39
1 | 219 | 28.52 | 37.15 | 36.61


[(0, 253, 32.94, 36.04, 31.39), (1, 219, 28.52, 37.15, 36.61)]

## d) Aggregate + HAVING (post-aggregation filter)

In [10]:
run_sql("""
SELECT
    outcome,
    COUNT(*) AS n,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM diabetes_patients), 2) AS pct_of_population,
    ROUND(AVG(glucose),2) AS avg_glucose
FROM diabetes_patients
GROUP BY outcome
HAVING avg_glucose > 120;
""")

SQL: SELECT
    outcome,
    COUNT(*) AS n,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM diabetes_patients), 2) AS pct_of_population,
    ROUND(AVG(glucose),2) AS avg_glucose
FROM diabetes_patients
GROUP BY outcome
HAVING avg_glucose > 120; 

Outcome | n | pct_of_population | avg_glucose
------------------------------------------------------------------------------------------------------------------------
1 | 268 | 34.9 | 141.26


[(1, 268, 34.9, 141.26)]

## e) JOIN with lookup table

In [11]:
run_sql("""
SELECT
    p.outcome,
    r.risk_label,
    COUNT(*) AS n,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM diabetes_patients), 2) AS pct_of_population,
    ROUND(AVG(p.glucose), 2) AS avg_glucose,
    ROUND(AVG(p.bmi), 2) AS avg_bmi
FROM diabetes_patients AS p
JOIN (
    SELECT 0 AS outcome, 'Low Risk' AS risk_label
    UNION ALL
    SELECT 1 AS outcome, 'High Risk' AS risk_label
) AS r
ON p.outcome = r.outcome
GROUP BY p.outcome, r.risk_label
ORDER BY n DESC;
""")

SQL: SELECT
    p.outcome,
    r.risk_label,
    COUNT(*) AS n,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM diabetes_patients), 2) AS pct_of_population,
    ROUND(AVG(p.glucose), 2) AS avg_glucose,
    ROUND(AVG(p.bmi), 2) AS avg_bmi
FROM diabetes_patients AS p
JOIN (
    SELECT 0 AS outcome, 'Low Risk' AS risk_label
    UNION ALL
    SELECT 1 AS outcome, 'High Risk' AS risk_label
) AS r
ON p.outcome = r.outcome
GROUP BY p.outcome, r.risk_label
ORDER BY n DESC; 

Outcome | risk_label | n | pct_of_population | avg_glucose | avg_bmi
------------------------------------------------------------------------------------------------------------------------
0 | Low Risk | 500 | 65.1 | 109.98 | 30.3
1 | High Risk | 268 | 34.9 | 141.26 | 35.14


[(0, 'Low Risk', 500, 65.1, 109.98, 30.3),
 (1, 'High Risk', 268, 34.9, 141.26, 35.14)]

## f) DERIVED TABLE - categorize by computed field

In [12]:
run_sql("""
SELECT 
    age_group,
    COUNT(*) AS n,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM diabetes_patients), 2) AS pct_of_population,
    ROUND(AVG(bmi), 2) AS avg_bmi,
    ROUND(AVG(glucose), 2) AS avg_glucose
FROM (
    SELECT *,
        CASE 
            WHEN age < 30 THEN 'Under 30'
            WHEN age BETWEEN 30 AND 50 THEN '30-50'
            ELSE 'Over 50'
        END AS age_group
    FROM diabetes_patients
) AS t
GROUP BY age_group
ORDER BY n DESC;
""")

SQL: SELECT 
    age_group,
    COUNT(*) AS n,
    ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM diabetes_patients), 2) AS pct_of_population,
    ROUND(AVG(bmi), 2) AS avg_bmi,
    ROUND(AVG(glucose), 2) AS avg_glucose
FROM (
    SELECT *,
        CASE 
            WHEN age < 30 THEN 'Under 30'
            WHEN age BETWEEN 30 AND 50 THEN '30-50'
            ELSE 'Over 50'
        END AS age_group
    FROM diabetes_patients
) AS t
GROUP BY age_group
ORDER BY n DESC; 

age_group | n | pct_of_population | avg_bmi | avg_glucose
------------------------------------------------------------------------------------------------------------------------
Under 30 | 396 | 51.56 | 31.39 | 113.74
30-50 | 291 | 37.89 | 33.31 | 125.4
Over 50 | 81 | 10.55 | 30.21 | 139.68


[('Under 30', 396, 51.56, 31.39, 113.74),
 ('30-50', 291, 37.89, 33.31, 125.4),
 ('Over 50', 81, 10.55, 30.21, 139.68)]

# Close db connection

In [13]:
close_db(conn)

🔒 Connection closed.
